In [1]:
# import and setup
# ------------------------------------------------

import pickle
from Config import SimpleConfig

config = SimpleConfig()

/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# Load sample prompt and sample data.
# ------------------------------------------------

# Load and tokenize the prompt
with open("manual_data_set/ExactSampleTrainSamplePrompt.txt", "r", encoding="utf-8") as f:
    raw_data_prompt = f.read()

# Process test cases from file.
with open("manual_data_set/ExactSampleTrainSample.txt", "r", encoding="utf-8") as f:
    raw_data = f.read()

In [3]:
lines = raw_data.split("\n")
testcases = []
heads = []
head = []
current_testcase = []
in_Test_target_object = False
in_Test_target = False
got_head = False
idx = 0
for line in lines:
    # print(line)
    line = line.strip()
    if line == "<Test Case>":
        current_testcase.append(config.breason)
        continue
    if line == "<Test Target Object>":
        in_Test_target_object = True
    
    if line == "<Test Target>":
        in_Test_target = True

    if line == "<Clang-repl Test>":
        current_testcase.append(config.ereason)
        current_testcase.append(config.banswer)

    if got_head == False and (in_Test_target_object or in_Test_target):
        head.append(line)
        
    if line == "</Test Case>":
        current_testcase.append(config.eanswer)
        testcases.append(current_testcase)
        heads.append(head)
        current_testcase = []
        head = []
        got_head = False
    else:
        current_testcase.append(line)
    
    if line == "</Test Target Object>":
        in_Test_target_object = False
        got_head = True
    if line == "</Test Target>":
        in_Test_target = False
        got_head = True


In [4]:
global_samples = []
samples = []
sample_heads = []

def check_eq(head1, head2):
    if head1 is None:
        return False
    if len(head1) != len(head2):
        return False
    for idx in range(len(head1)):
        if head1[idx] != head2[idx]:
            return False
    return True

last_head = None
# Tokenize each test case
for idx in range(len(testcases)):
    testcase = testcases[idx]
    text_case_text = "\n".join(testcase).strip() + '\n'
    if check_eq(last_head, heads[idx]):
        samples[-1] = samples[-1] + "\n<Need More Test/>\n" + text_case_text
    else:
        samples.append(text_case_text)
        sample_heads.append("\n".join(heads[idx]))
    last_head = heads[idx]

#samples

In [5]:
# from https://colab.research.google.com/github/TrelisResearch/llama-2-setup/blob/main/Llama_2_Prompt_and_Tokenizer_Format.ipynb#scrollTo=sQ4dBAJOovzz
#
# prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}\n\n"
# inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

# <s> [INST] <<SYS>>
# You are a helpful assistant.
# <</SYS>>
#
# Howdy! [/INST]
#
# Well, howdy there! *adjusts cowboy hat* It's a pleasure to meet you! How can I help you today? Do you have any questions or tasks you'd like me to assist you with? Just let me

# use startcoder
#         content = f"### Instruction\n\n{q}\n### Response\n\n{a}\n"


from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(config.model_full_name)

# For each sample, concatenate prompt tokens, sample tokens, and pad tokens for both fields.
for i in range(len(samples)):
    _sample = samples[i]
    # config.bos + << automatically added to start
    sample_front = "### Instruction\n\n" + sample_heads[i] + "\nWrtie a Clang-repl Test\n"
    sample = sample_front \
        + "\n### Response\n\n" \
        + _sample
    samples[i] = sample

# [print(sample) for sample in samples]
tokenizer(sample_front, return_tensors="pt")['input_ids'].shape

torch.Size([1, 86])

In [6]:
import pickle
from Config import SimpleConfig

config = SimpleConfig()
with open(config.dataset_file, "wb") as f:
    pickle.dump(samples, f)


In [7]:


for _sample in samples:
    sample_token = tokenizer(_sample, return_tensors="pt")
    input_ids = sample_token['input_ids'][0]
    global_samples.append(input_ids)

In [8]:
global_samples
global_samples = [atensor.numpy() for atensor in global_samples]

In [9]:
import pickle
from Config import SimpleConfig

config = SimpleConfig()
#with open(config.dataset_file, "wb") as f:
#    pickle.dump(global_samples, f)
#global_samples

In [10]:
tokenizer(config.esys, return_tensors="pt")

{'input_ids': tensor([[  222,    65,   396, 12513,  2023,   499]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [11]:
for i in range(len(global_samples[0])):
    print(global_samples[0][i])
    if config.esys == global_samples[0][i]:
        print(i)
        break
    

1502
21052
222
222
65
1024
10069
2210
67
222
1121
3161
18789
51
480
461
341
3668
51
222
396
1024
10069
2210
67
222
92
2069
1093
331
409
1600
50
22101
2128
222
222
1502
5178
222
222
96
36923
98
222
65
1024
10069
2210
67
222
1121
3161
18789
51
480
461
341
3668
51
222
396
1024
10069
2210
67
222
65
1024
10069
67
222
429
1035
45
429
331
49
648
342
46
320
222
620
331
494
342
64
222
130
222
396
1024
10069
67
222
65
1024
2210
67
222
10574
708
1035
3235
341
3831
3668
51
222
396
1024
2210
67
222
65
1630
2426
67
222
11978
648
331
299
244
57
64
222
11978
648
342
299
244
59
64
222
396
1630
2426
67
222
65
7705
5601
67
222
11978
925
3586
1074
630
244
54
53
64
222
1527
222
396
7705
5601
67
222
36212
36923
98
222
96
946
5696
546
98
222
65
72
1600
50
22101
2128
67
222
11978
453
6566
9925
63
222
11978
222
11978
453
2128
2210
63
12082
708
1035
3235
341
3831
3668
51
222
11978
453
2128
12029
63
2242
2466
1024
222
11978
648
331
299
244
57
64
222
11978
648
342
299
244
59
64
222
11978
648
1074
299
1035
45
102
